# 1. <a id='toc1_'></a>[Import packages](#toc0_)

First, we import the necessary packages for this exercise.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [ ]:
# Change to directory where data is stored
data_dir = "/kyukon/data/gent/courses/2025/computphys_C004504/input"
# For people following the old course: "/kyukon/data/gent/courses/2025/computfys_C001827/input"

# 2. <a id='toc2_'></a>[Open simulation data](#toc0_)

In this exercise we will take a look at the precipitation event from July 2021 which caused massive flooding in the Vesder basin. Some more information about this event can be found [here](https://www.vrt.be/vrtnws/nl/2022/07/08/tijdslijn/) (in Dutch) and [here](https://en.wikipedia.org/wiki/2021_European_floods) (in English). We have simulated this case in several different configurations. All the output data can be found in the 'data' folder: `/data/gent/courses/2025/computphys_C004504/input`. This data is stored in netCDF-files, which can be easily read as xarray Datasets (see introductory notebook on xarray). The naming convention of the netCDF-files is outlined below. You will need to understand it to solve some exercises.

<details><summary>netCDF-file naming convention</summary>

Each of the netCDF-files has a structured name `<configuration>_<starttime>_<nhours>_<variable>_<tstart>_<tstop>_<tstep>_regridded.nc` where the different elements have the following meaning:
1. `<configuration>`: This refers to the specific set-up of the model. For some experiments, we can change certain parameters in the model and investigate their impact. For this exercise, however, we will only use the data from the 'baseline' simulation.
2. `<starttime>`: This refers to the start time of the simulation. By varying the start time, we can create an ensemble of different simulations of the same event (see below). The start time is written in the format of `<year><month><day><hour>`.
3. `<nhours>`: This refers to the number of hours that the simulation has run.
4. `<variable>`: This refers to the variable saved in this file. We use the standard abbreviations from the Climate and Forecast (CF) Metadata Conventions. The ones you will mostly need in these exercises are:
    - *pr*: Precipitation
    - *tas*: Near-Surface Air Temperature
5. `<tstart>`: This refers to the first time step of the data in the file. This is written in the format of `<year>-<month>-<day>T<hour>`.
6. `<tstart>`: This refers to the last time step of the data in the file. This is written in the format of `<year>-<month>-<day>T<hour>`.
7. `<tstep>`: This refers to the time between subsequent time steps in the data, expressed in seconds. You will see that this is 3600 s for all files (so all data have an hourly resolution).
8. `regridded`: This refers to the fact that the data has been regridded from the orginal coordinate reference system (CRS) to a latitude-longitude reference system. This makes the data easier to work with.
9. `.nc`: This extension denotes the fact that this is a netCDF-file.

</details>

Let's now start by opening a file and loading in the data! Load in the precipitation data (with variable name `pr`) corresponding to the `baseline` simulation which started at midnight of 1 July 2021 and ran for 384 hours. Following the naming convention this file corresponds to `baseline_2021070100_384_pr_2021-07-01T01_2021-07-17T00_3600_regridded.nc`. You can use the xarray function `open_dataset` ([documentation](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html)).

**Tip**: you can run commands in the terminal by putting an exclamation mark at the beginning of a notebook cell. For example, you can use `!ls "/data/gent/courses/2025/computphys_C004504/input"` to inspect the contents of the directory.

Load in this dataset as the Python variable `ds` and inspect its contents by printing `ds` or simply running a cell which ends with `ds`. The following questions/tasks will help you along:
- What are the dimensions of the data? Where can you find information about these dimensions?
- First consider the spatial dimensions. What are the units? What is the resolution? Calculate the resolution in (kilo)metres. Use the fact that the radius of the Earth is 6371 km.
- Next, consider the attributes of the variable `pr`. What are its units? Usually, precipitation flux is expressed in mm/h. Convert the units in the file (if necessary) to $\text{mm}/\text{h}$ by using the function `convert_to_mm_h`. You need to define the conversion factor `a`. Use the fact that water has a density of 1000 $\text{kg}/\text{m}^3$ in your calculations.
- Lastly, in the attributes of the variable `pr`, you can find `cell_methods = time: mean`. This signifies that the variable does not represent an instantaneous value at a certain moment in time, but rather provides an average value over a certain time interval. The bounds of this time interval are given in the coordinate `time_bnds`. So, every value of the coordinate `time` has two corresponding values in the coordinate `time_bnds`: the start and end time of the interval over which the average is taken. Compare the values in `time_bnds` to those in `time`. For a `time` value at moment $x$, which interval corresponds to this? What is the advantage of providing the average precipitation flux, instead of the instantaneous value?


<details><summary>Hints</summary>

- When running a cell, which ends with `ds`, a nice overview of the Dataset is provided. The dimensions are shown in the top row. Alternatively, `ds.dims` can be used to print the information.
- The units of certain coordinates (`.coords`) can be found in the attributes (`.attrs`).
- The resolution is the difference between subsequent coordinate values.
- The resolution in kilometres can be calculated by using formulas for a spherical coordinate system: [Wikipedia link](https://en.wikipedia.org/wiki/Spherical_coordinate_system)
- The units of certain variable can be found in the attributes (`.attrs`).
- Convert the units from mass to volume by means of the mass density of water.
- Updating the values of a DataArray in-place can be tricky. Either define a new variable `pr_mm_h` to contain the values in mm/h or update the values of the NumPy array in-place by calling `.values`. In either case, don't forget to define/update the `units` attribute for your bookkeeping!
- Look at a coordinate value for a specifix index (e.g. 0) for both `time` and `time_bnds`. What is the link between the two?

</details>

In [ ]:
def convert_to_mm_h(ds, a):
    """
    a: conversion factor to mm/h
    """

    if "pr" in ds:
        ds.pr.values = ds.pr.values * a
        ds.pr.attrs["units"] = "mm/h"

    return ds

# 3. <a id='toc3_'></a>[Plot the accumulated precipitation](#toc0_)

Let's now focus on the period of the Vesder event. In this part you will make a plot of the accumulated precipitation (expressed in $\text{mm}$) over the period between 13 July 2021 06:00 and 15 July 2021 06:00. This is the period during which most of the precipitation fell during the event. You can use the following code to make your plot (but of course you will have to make changes!). This code shows you how to plot the data on a geographical map with coastlines and borders.

- Calculate the accumulated precipitation (in $\text{mm}$) for the period between 13 July 2021 06:00 and 15 July 2021 06:00.
- Make a plot of this quantity and inspect the plot. 
- Where did most of the precipitation fall? What is the maximum value of accumulated precipitation during this period? Compare these values with normal climatological values. For the Dutch (and French) speakers, you can use the following webpage of the RMI (KMI): [klimaatatlas](https://www.meteo.be/nl/klimaat/klimaat-van-belgie/klimaatatlas).

<details><summary>Hints</summary>

No extra hints for this exercise! The skeleton code is the hint ;)

</details>

In [ ]:
# We define some dummy data with the same dimensions to plot. The data is filled with ones.
# For your plot, you of course have to use the simulation data!
da_dummy = xr.DataArray(
    data=np.ones((70, 75)),
    dims=("lat", "lon"),
    coords={"lon": np.arange(2.0, 7.2, 0.07), "lat": np.arange(49.0, 52.11, 0.045)},
)

# Change the values of these levels to see what they define
levels = np.arange(0.85, 1.16, 0.1)

# Create a figure.
# The projection keyword tells matplotlib (and Cartopy) that we are plotting geographical data.
# The ccrs.PlateCarree() then denotes that the data is in lat-lon format.
fig, ax = plt.subplots(layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()})
dummy_plot = da_dummy.plot(ax=ax, x="lon", y="lat", levels=levels, add_colorbar=False)

# Add a title to your plot
ax.set_title("Title of plot")

# This adds coastlines and borders to the plot.
ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")

# This aspect-ratio controls the ratio of the x- and y-axes.
# Try and find a value which provides a nice picture of our country.
# (Use the information from above about the spatial resolution and the values in kilometres)
aspect_ratio = 1.0
ax.set_aspect(aspect_ratio)

# Add a color bar and a title to the figure
fig.colorbar(dummy_plot, label="Colorbar label")
fig.suptitle("Title of figure", fontsize="large")

# Show the plot!
plt.show()

# 4. <a id='toc4_'></a>[Compare with observational gridded data](#toc0_)

We will now compare the simulation data with observational data. As observational data, we take the CLIMATE-GRID dataset of the RMI. This is a gridded dataset with a resolution of 5 km. It contains only daily values. For precipitation, the value on a certain day represents accumulated precipitation between 8:00 on the day in question and 8:00 on the following day. 

> **IMPORTANT**: It is important to note that the CLIMATE_GRID uses local time (LT), which is CEST (Central European Summer Time) in the summer. There is a difference between summer and winter, because of daylight saving time ("zomertijd"). On the other hand, the time in the simulation is UTC (Universal Time Coordinated). We can convert from CEST to UTC by subtracting 2 hours. So, 8:00 CEST corresponds to 6:00 UTC. (That is why this period has been chosen for the previous parts of this exercise.) 

Follow these steps to compare these observations with the simulation data:
- Load in the CLIMATE-GRID data from the file `CLIMATE_GRID_pr_2021_07_daily_latlon_5km.nc`. Inspect the attributes of the dataset and of the variable `pr`. Convert the units to $\text{mm}/\text{h}$ (if necessary). How can you relate these values to daily accumulated precipitation (expressed in $\text{mm}$)?
- Select the precipitation data corresponding to the period between 13 July 2021 08:00 LT / 06:00 UTC and 15 July 2021 08:00 LT  and plot the accumulated precipitation. Use the same color scheme as for the simulation data. What is the maximum value for the observations? When comparing the figures of the simulation and the observations by eye, what differences do you see?
- Compare the simulations directly with the observations by plotting the bias (difference) in accumulated precipitation over this period. What do you see on this figure?
- Calculate and compare the domain-averaged precipitation values. What can you conclude?

<details><summary>Hints</summary>

- Loading a DataSet can be done with the `open_dataset` function.
- How are values in $\text{mm}/\text{h}$ related to values in $\text{mm}$? An nice analogues is speed ($\text{km}/\text{h}$) and distance ($\text{km}$).
- Selecting data can be done with the `.sel` function. 
- Using the same color scheme as for the simulation data can be done by defining the same `levels` for plotting. 
- You can extend your colorbar using the `extend` keyword argument with `extend="min"` or `extend="max"` or `extend="both"`
- You can calculate the difference between two xarray DataSets with identical dimensions simply by subtracting them from each other.
- A domain-average is a `mean` over the domain dimensions `lat` and `lon`. 

</details>

# 5. <a id='toc5_'></a>[Compare with observational station data](#toc0_)

We will now take a look at some time series data of the event. For this exercise, you will use station measurements from the five stations which measured the highest precipitation amounts during the Vesder event. These station data are hourly values, contrary to the daily CLIMATE-GRID data. The stations employ the local time (CEST).

- Load in the station data for the five stations from the file `SETHY_pr_2021-07-12T01_2021-07-17T00.nc`. Inspect the structure of this Dataset. Convert (as usual) the precipitation data to $\text{mm}/\text{h}$, if necessary. 
- Make a plot of the locations of the stations on a map. You can use the function `.plot.scatter()`, which you can apply to a Dataset. This function accepts arguments `x` and `y`, which tell the function which Dataset variables/coordinates to use as x- and y-values for the plot. Where are the five stations located? Does it make sense that these stations received the highest precipitation amounts in light of the CLIMATE-GRID data (look at your previous plots)?
- For each of the five stations, plot the time series of hourly precipitation flux for the time period of the event (i.e., precipitation from 13 July 2021 08:00 LT until 15 July 2021 08:00 LT). Also, make a plot of the cumulative precipitation amount over time. What can you conclude when comparing these station observations to the simulated data? Is this in line with previous conclusions? 
- Finally, compare the station measurements to the CLIMATE-GRID data. What do you see?

<details><summary>Hints</summary>

- Loading a DataSet can be done with the `open_dataset` function.
- The DataSet structure can be inspected by outputting the DataSet. As this DataSet contains station observations in several points, the dimensions will be different.
- The `lon` and `lat` values can be plotted in a scatter plot by using `x=` and `y=`. Does `lon` (`lat`) correspond to `x` or `y`?
- When making your Matplotlib figure use `subplot_kw={"projection": ccrs.PlateCarree()}` to define a Geographical Coordinate System. Then use `ax.coastlines(alpha=0.7)` to automatically add coastlines to your figure.
- Use `hue` to make several line plots in the same function. 
- Cumulative precipitaiton is the cumulative sum `.cumsum()` over `time`.
- Comparing station data with the corresponding CLIMATE-GRID data needs to be done point-wise by extracting the nearest points from the CLIMATE-GRID data for each station.

</details>

# 6. <a id='toc6_'></a>[Work with ensemble data](#toc0_)

In this exercise we will look not at one simulation but at an ensemble of different simulations. Ensembles consist of several simulations for the same period, which differ slightly in certain aspects. With an ensemble we can gauge the uncertainty of the simulation. Ensembles can be created in several ways, for example: by slightly varying the initial conditions, by slightly changing certain parameters in the model, or simply by using different models. We have created an ensemble by running the baseline simulation, but starting it at different times. This slight change in 'initial conditions' is enough to change the course of the simulation even though the same model set-up and identical boundary conditions are used.

You will now study this ensemble with the help of these steps:
- Load in the precipitation data of an ensemble created by running the `baseline` simulation with different start times and save it as `ds_ens`. These start times vary from 2021-07-01T00 to 2021-07-04T00 with steps of 12 hours. In total, the ensemble has 7 members.
- Calculate the bias of the domain-averaged accumulated precipitation during the period between 13 July 2021 08:00 LT / 06:00 UTC and 15 July 2021 08:00 LT / 06:00 UTC for each member of the ensemble compared to the CLIMATE-GRID data. Repeat this exercise but for the two 24-hour periods inside of this 48-hour period. Create a plot to help you visualise these values. What can you conclude?
- Plot the ensemble data for each of the station locations. What do you see?

<details><summary>Hints</summary>

- The ensemble has an extra dimension corresponding to the specific member of the ensemble. As each member in our ensemble differs in starting time, this coordinate is called `rstart` (from run-start).
- The domain-average can be used by calculating the `.mean()` over the domain dimensions `lat` and `lon`. What are the dimensions of the resulting xarray Dataset?
- A period can be selected with the `.sel()` function.
- The difference for each member can be plotted in one plot by using `hue` and the `rstart`-coordinate.
- Extracting the data at station locations can be done with `.sel(lat= , lon=)`

</details>

In [ ]:
run_name = "baseline"
run_info_list = [
    ("2021070100", 384),
    ("2021070112", 372),
    ("2021070200", 360),
    ("2021070212", 348),
    ("2021070300", 336),
    ("2021070312", 324),
    ("2021070400", 312),
]

ds_ens_list = []

for run_info in run_info_list:
    rstart, nhours = run_info
    filepath = f"{data_dir}/{run_name}_{rstart}_{nhours}_pr_*.nc"
    ds_ens = xr.open_mfdataset(filepath, engine="netcdf4", chunks="auto")
    ds_ens_list.append(ds_ens)

ds_ens = xr.concat(ds_ens_list, dim="rstart")
# ds_ens = convert_to_mm_h(ds_ens, a=)
ds_ens.load()

# 7. <a id='toc7_'></a>[Extra. RMI warning levels](#toc0_)

We will now consider the severity of the event from a different angle. The Royal Meteorological Institute of Belgium (RMI or KMI in Dutch) issues color-coded warning levels to warn the public about an upcoming extreme events. The colors for these warnings are yellow, orange and red. Warnings exist for all types of events, such as heatwaves, wind and thunder storms, but also for precipitation/rain! Certain thresholds are used by the institute to assign a color to an event. Information about these thresholds can be found on [warnings](https://www.meteo.be/nl/weer/waarschuwingen/weerflashes) (you can change the language in the upper right corner of the page).

Investigate the simulated precipitation during the period between 13 July 2021 06:00 UTC and 15 July 2021 06:00 UTC and determine at which points a yellow/orange/red warning for rain would be issued at any time during this period based on this simulation data. The following functions will probably be helpful for this exercise:
- the `rolling` function: [documentation](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.rolling.html#xarray.DataArray.rolling)
- the `where` function: [documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.where.html#xarray.Dataset.where)

Based on this simulation data, for which parts of the country would a red warning be issued?